In [26]:
import pandas as pd
from nbapy import game, shot_chart, player, scoreboard, team
import datetime

In [27]:
def datascrape(start):
    start = datetime.datetime.strptime(start, "%d-%m-%Y")
    #end = datetime.datetime.strptime("21-07-2014", "%d-%m-%Y")
    end = datetime.datetime.today()
    dates = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
    df = pd.DataFrame({"date": dates})
    df["date"] = df["date"].astype(str)
    df[['Year', 'Month', "Day"]] = df["date"].str.split('-', 2, expand=True)
    df["Month"] = df["Month"].str.lstrip('0')
    df["Day"] = df["Day"].str.lstrip('0')
    
    #to limit to five
    #df = df.head()
    dates = []
    for i, row in df.iterrows():
        day = {}
        day["year"] = row["Year"]
        day["month"] = row["Month"]
        day["day"] = row["Day"]
        dates.append(day)
    
    init = pd.DataFrame({
    "GAME_DATE_EST_x": [],
    "GAME_ID": [],
    "TEAM_ABBREVIATION_x": [],
    "TEAM_ABBREVIATION_y": [],
    "PTS_x": [],
    "PTS_y": []
    })
    
    for date in dates:
        scores = scoreboard.Scoreboard(month=int(date["month"]), day=int(date["day"]), year=int(date["year"]), league_id='00', offset=0)
        df = scores.line_score()
        if df.empty:
            pass
        else:
            df_1 = df.drop_duplicates(subset=['GAME_ID'], keep='first')
            df_2 = df.drop_duplicates(subset=['GAME_ID'], keep='last')
            merged = df_1.merge(df_2, on=["GAME_ID"])
            merge = merged[["GAME_DATE_EST_x", "GAME_ID", "TEAM_ABBREVIATION_x", "TEAM_ABBREVIATION_y", "PTS_x", "PTS_y"]]
            init = pd.concat([init, merge])
    
    return init

In [33]:
df = datascrape("20-12-2020")

In [34]:
#Rename columns
df = df.rename(columns={
            "GAME_DATE_EST_x": "date",
            "GAME_ID": "game_id",
            "TEAM_ABBREVIATION_x": "away",
            "TEAM_ABBREVIATION_y": "home",
            "PTS_x": "away_pts",
            "PTS_y": "home_pts"})

In [35]:
#Add catagorical for win
df.loc[df["home_pts"] > df["away_pts"], "home_win"] = 1
df.loc[df["home_pts"] < df["away_pts"], "home_win"] = 0

In [36]:
df

,date,game_id,away,home,away_pts,home_pts,home_win
0,2020-12-22T00:00:00,0022000001,GSW,BKN,99.0,125.0,1.0
1,2020-12-22T00:00:00,0022000002,LAC,LAL,116.0,109.0,0.0
0,2020-12-23T00:00:00,0022000010,CHA,CLE,114.0,121.0,1.0
1,2020-12-23T00:00:00,0022000011,NYK,IND,107.0,121.0,1.0
2,2020-12-23T00:00:00,0022000012,MIA,ORL,107.0,113.0,1.0
...,...,...,...,...,...,...,...
4,2021-03-14T00:00:00,0022000597,CLE,ATL,82.0,100.0,1.0
5,2021-03-14T00:00:00,0022000598,BOS,HOU,134.0,107.0,0.0
6,2021-03-14T00:00:00,0022000599,POR,MIN,112.0,114.0,1.0
7,2021-03-14T00:00:00,0022000600,TOR,CHI,95.0,118.0,1.0
